<a href="https://colab.research.google.com/github/sgokulakannankg19/ClinicalTrialFiles/blob/main/Report_Exact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gspread

In [ ]:
!pip install gspread-formatting

In [80]:
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import requests
import re
import time
from datetime import date

import gspread

from gspread_formatting import *

from oauth2client.client import GoogleCredentials
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# gc = gspread.authorize(GoogleCredentials.get_application_default())

In [81]:
def get_gene_aliases_from_master_genes(gene):
  gene = gene
  wb_master_gene = gc.open_by_key("1KvrHR3ShutWfOHvXYvyWn44B4FggKvA2b9Wy8RSad1E")

  ws_compilations = wb_master_gene.worksheet('COMPILATION')
  rows = ws_compilations.get_all_values()

  df_compilations = pd.DataFrame(rows[1:], columns=rows[0])

  new_df = df_compilations.set_index('Gene')
  gene_name_results = new_df.loc[gene, 'Aliases-Final List': 'SEARCH TERMS']

  gene_names = []

  gene_name_alias_list = gene_name_results['Aliases-Final List'].split('|')
  gene_name_searchterms = gene_name_results['SEARCH TERMS'].split('|')

  gene_names = gene_name_alias_list + gene_name_searchterms
  gene_names = [gene.strip() for gene in gene_names]
  
  if "" in gene_names:
    gene_names.remove("")
  
  return gene_names

In [82]:
def find_cancer_nct_id(final_combined_df):
  result_list = [] 
  for index in final_combined_df.index: 
  
    trial_rows=[]
    cancer_found_in = []
    cancers_found_list = []
    
    
    data_df = final_combined_df.loc[index]

    nct_id = data_df['NCT ID']
    study_type = data_df['Study Type']
    overall_status = data_df['Recruitment Status']
    phase = data_df['Phase']
    brief_title = data_df['Brief Title']
    official_title = data_df['Official Title']
    brief_summary = data_df['Brief Summary']
    detailed_description = data_df['Detailed Description']
    cancer_type = data_df['Cancer Type']
    study_purpose = data_df['Study Purpose']
    intervention = data_df['Intervention']
    intervention_description = data_df['Intervention Description']
    intervention_other_name = data_df['Intervention Other Name']
    study_model = data_df['Study Model']
    arm_title = data_df['Arm Title']
    arm_description = data_df['Arm Description']
    arm_group_intervention = data_df['Arm Group Intervention']
    eligibility_criteria = data_df['Eligibility Criteria']
    inclusion_criteria = data_df['Inclusion Criteria']
    exclusion_criteria = data_df['Exclusion Criteria']
    keywords = data_df['Keywords']
    pmid = data_df['PMID']
    url = data_df['URL']
    primary_om_title = data_df['Primary OM Title']
    primary_om_description = data_df['Primary OM Description']
    secondary_om_title = data_df['Secondary OM Title']
    secondary_om_description = data_df['Secondary OM Description']
    other_om_title = data_df['Other OM Title']
    other_om_description = data_df['Other OM Description']
    mesh_terms = data_df['MeSH Terms']
    character_count = data_df['Character Count']

    cancer_list = ["cancer", "neoplasm", "tumor", "malignancy", "oncology", "neoplasia", "neoplastic syndrome", "neoplastic disease"]

    column_list = ['study_type', 'phase', 'brief_title', 'official_title', 'brief_summary', 'detailed_description', 'cancer_type', 'study_purpose', 'intervention', 'intervention_description', 'intervention_other_name', 'study_model', 'arm_title', 'arm_description', 'arm_group_intervention', 'eligibility_criteria', 'primary_om_title', 'primary_om_description', 'secondary_om_title', 'secondary_om_description', 'other_om_title', 'other_om_description', 'mesh_terms']


    for cancer in cancer_list:
      cancer = cancer.lower()
      if (re.findall(r"\b{}\b".format(cancer), study_type.lower())):
        cancer_found_in.append(f"{cancer} = Study Type")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), brief_title.lower())):
        cancer_found_in.append(f"{cancer} = Brief Title")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), official_title.lower())):
        cancer_found_in.append(f"{cancer} = Official Title")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), brief_summary.lower())):
        cancer_found_in.append(f"{cancer} = Brief Summary")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), detailed_description.lower())):
        cancer_found_in.append(f"{cancer} = Detailed Description")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), cancer_type.lower())):
        cancer_found_in.append(f"{cancer} = Cancer Type")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), intervention_description.lower())):
        cancer_found_in.append(f"{cancer} = Intervention Description")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), intervention_other_name.lower())):
        cancer_found_in.append(f"{cancer} = Intervention Other Name")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), arm_description.lower())):
        cancer_found_in.append(f"{cancer} = Arm Description")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), eligibility_criteria.lower())):
        cancer_found_in.append(f"{cancer} = Eligibility Criteria")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), inclusion_criteria.lower())):
        cancer_found_in.append(f"{cancer} = Inclusion Criteria")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), exclusion_criteria.lower())):
        cancer_found_in.append(f"{cancer} = Exclusion Criteria")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), keywords.lower())):
        cancer_found_in.append(f"{cancer} = Keywords")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), primary_om_title.lower())):
        cancer_found_in.append(f"{cancer} = Primary OM Title")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), primary_om_description.lower())):
        cancer_found_in.append(f"{cancer} = Primary OM Description")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), secondary_om_title.lower())):
        cancer_found_in.append(f"{cancer} = Secondary OM Title")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), secondary_om_description.lower())):
        cancer_found_in.append(f"{cancer} = Secondary OM Description")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), other_om_title.lower())):
        cancer_found_in.append(f"{cancer} = Other OM Title")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), other_om_description.lower())):
        cancer_found_in.append(f"{cancer} = Other OM Description")
        cancers_found_list.append(cancer)
      if (re.findall(r"\b{}\b".format(cancer), mesh_terms.lower())):
        cancer_found_in.append(f"{cancer} = MeSH Terms")
        cancers_found_list.append(cancer)

In [84]:
def filter_gene_found_in(sheet_df, gene_names):

  # Sheet for drug pathway sheet
  
  # target_pathway_sh = gc.open_by_key("1_2AUMO9fLDVWoBF-DNgw6iBzr4uzwsbV6mJv-KzP1Yk")
  # target_pathway_sheet = target_pathway_sh.worksheet("PIK3CA")
  # target_pathway_rows = target_pathway_sheet.get_all_values()

  # target_pathway_df = pd.DataFrame(target_pathway_rows[1:], columns=target_pathway_rows[0])
  
  # ----------------------------------------------------------------------------------------------


  # Sheet for variant

  # variant_sh = gc.open_by_key("1_hKejRNMKYEf4jPQYSvzJZZUHIm-eNDDcuyc8LB6L5M")
  # variant_sheet = variant_sh.worksheet('Oncogene')
  # variant_rows = variant_sheet.get_all_values()

  # variant_df = pd.DataFrame(variant_rows[1:], columns=variant_rows[0])

  # data = variant_df.loc[(variant_df.Gene == gene_name)]
  # variant_names = data['Variant'].to_numpy().tolist()
  
  # -------------------------------------------------------------------------------------------------

  result_list = [] 
  for index in sheet_df.index: 
  
    trial_rows=[]
    gene_found_in = []
    genes_found_list = []
    variant_found_in = []
    variants_found_list = []
    mutant_found_in = []
    wild_type_found_in = []
    
    data_df = sheet_df.loc[index]
  
    
    nct_id = data_df['NCT ID']
    study_type = data_df['Study Type']
    overall_status = data_df['Recruitment Status']
    phase = data_df['Phase']
    brief_title = data_df['Brief Title']
    official_title = data_df['Official Title']
    brief_summary = data_df['Brief Summary']
    detailed_description = data_df['Detailed Description']
    cancer_type = data_df['Cancer Type']
    study_purpose = data_df['Study Purpose']
    intervention = data_df['Intervention']
    intervention_description = data_df['Intervention Description']
    intervention_other_name = data_df['Intervention Other Name']
    study_model = data_df['Study Model']
    arm_title = data_df['Arm Title']
    arm_description = data_df['Arm Description']
    arm_group_intervention = data_df['Arm Group Intervention']
    eligibility_criteria = data_df['Eligibility Criteria']
    inclusion_criteria = data_df['Inclusion Criteria']
    exclusion_criteria = data_df['Exclusion Criteria']
    keywords = data_df['Keywords']
    pmid = data_df['PMID']
    url = data_df['URL']
    primary_om_title = data_df['Primary OM Title']
    primary_om_description = data_df['Primary OM Description']
    secondary_om_title = data_df['Secondary OM Title']
    secondary_om_description = data_df['Secondary OM Description']
    other_om_title = data_df['Other OM Title']
    other_om_description = data_df['Other OM Description']
    mesh_terms = data_df['MeSH Terms']
    character_count = data_df['Character Count']

    # ---------------------------------------FOR GENE FOUND IN LIST ---------------------------------------

    for gene in gene_names:
      gene = gene.lower()
      if (re.findall(r"\b{}\b".format(gene), brief_title.lower())):
        gene_found_in.append(f"{gene} = Brief Title")
        genes_found_list.append(gene)
      if (re.findall(r"\b{}\b".format(gene), official_title.lower())):
        gene_found_in.append(f"{gene} = Official Title")
        genes_found_list.append(gene)
      if (re.findall(r"\b{}\b".format(gene), brief_summary.lower())):
        gene_found_in.append(f"{gene} = Brief Summary")
        genes_found_list.append(gene)
      if (re.findall(r"\b{}\b".format(gene), detailed_description.lower())):
        # print("found", "Detailed Description")
        gene_found_in.append(f"{gene} = Detailed Description")
        genes_found_list.append(gene)
      if (re.findall(r"\b{}\b".format(gene), cancer_type.lower())):
        # print("found", "Cancer Type")
        gene_found_in.append(f"{gene} = Cancer Type")
        genes_found_list.append(gene)
      if (re.findall(r"\b{}\b".format(gene), intervention_description.lower())):
        # print("found", "Intervention Description")
        gene_found_in.append(f"{gene} = Intervention Description")
        genes_found_list.append(gene)
      if (re.findall(r"\b{}\b".format(gene), arm_title.lower())):
        # print("found", "Arm Title")
        gene_found_in.append(f"{gene} = Arm Title")
        genes_found_list.append(gene)
      if (re.findall(r"\b{}\b".format(gene), arm_description.lower())):
        # print("found", "Arm Description")
        gene_found_in.append(f"{gene} = Arm Description")
        genes_found_list.append(gene)
      if (re.findall(r"\b{}\b".format(gene), eligibility_criteria.lower())):
        # print("found", "Eligibility Criterial")
        gene_found_in.append(f"{gene} = Eligibility Criteria")
        genes_found_list.append(gene)
      if (re.findall(r"\b{}\b".format(gene), inclusion_criteria.lower())):
        # print("found", "Inclusion Criteria")
        gene_found_in.append(f"{gene} = Inclusion Criteria")
        genes_found_list.append(gene)
      if (re.findall(r"\b{}\b".format(gene), exclusion_criteria.lower())):
        # print("found", "Exclusion Criteria")
        gene_found_in.append(f"{gene} = Exclusion Criteria")
        genes_found_list.append(gene)
      if (re.findall(r"\b{}\b".format(gene), keywords.lower())):
        # print("found", "Keywords")
        gene_found_in.append(f"{gene} = Keywords")
        genes_found_list.append(gene)
      if (re.findall(r"\b{}\b".format(gene), primary_om_title.lower())):
        # print("found", "Primary OM Title")
        gene_found_in.append(f"{gene} = Primary OM Title")
        genes_found_list.append(gene)
      if (re.findall(r"\b{}\b".format(gene), primary_om_description.lower())):
        # print("found", "Primary OM Description")
        gene_found_in.append(f"{gene} = Primary OM Description")
        genes_found_list.append(gene)
      if (re.findall(r"\b{}\b".format(gene), secondary_om_title.lower())):
        # print("found", "Secondary OM Title")
        gene_found_in.append(f"{gene} = Secondary OM Title")
        genes_found_list.append(gene)
      if (re.findall(r"\b{}\b".format(gene), secondary_om_description.lower())):
        # print("found", "Secondary OM Description")
        gene_found_in.append(f"{gene} = Secondary OM Description")
        genes_found_list.append(gene)
      if (re.findall(r"\b{}\b".format(gene), other_om_title.lower())):
        # print("found", "Other OM Title")
        gene_found_in.append(f"{gene} = Other OM Title")
        genes_found_list.append(gene)
      if (re.findall(r"\b{}\b".format(gene), other_om_description.lower())):
        # print("found", "Other OM Description")
        gene_found_in.append(f"{gene} = Other OM Description")
        genes_found_list.append(gene)
      if (re.findall(r"\b{}\b".format(gene), mesh_terms.lower())):
        # print("found", "MeSH Terms")
        gene_found_in.append(f"{gene} = MeSH Terms")
        genes_found_list.append(gene)


    gene_found_in_string = " | ".join(gene_found_in)
    gene_found_in_list = gene_found_in_string.split(" | ")

    genes_found_list = list(set(genes_found_list))
    genes_found_string = " | ".join(genes_found_list)

    # print("gene found in completed")


    # ------------------------------------For variant list ----------------------------------------------

    # for variant in variant_names:
    #   variant = variant.lower()
    #   if (re.findall(r"\b{}\b".format(variant), brief_title.lower())):
    #     # print("found", "Brief Title")
    #     variant_found_in.append(f"{variant} = Brief Title")
    #     variants_found_list.append(variant)
    #   if (re.findall(r"\b{}\b".format(variant), official_title.lower())):
    #     # print("found", "Official Title")
    #     variant_found_in.append(f"{variant} = Official Title")
    #     variants_found_list.append(variant)
    #   if (re.findall(r"\b{}\b".format(variant), brief_summary.lower())):
    #     # print("found", "Brief Summary")
    #     variant_found_in.append(f"{variant} = Brief Summary")
    #     variants_found_list.append(variant)
    #   if (re.findall(r"\b{}\b".format(variant), detailed_description.lower())):
    #     # print("found", "Detailed Description")
    #     variant_found_in.append(f"{variant} = Detailed Description")
    #     variants_found_list.append(variant)
    #   if (re.findall(r"\b{}\b".format(variant), cancer_type.lower())):
    #     # print("found", "Cancer Type")
    #     variant_found_in.append(f"{variant} = Cancer Type")
    #     variants_found_list.append(variant)
    #   if (re.findall(r"\b{}\b".format(variant), intervention_description.lower())):
    #     # print("found", "Intervention Description")
    #     variant_found_in.append(f"{variant} = Intervention Description")
    #     variants_found_list.append(variant)
    #   if (re.findall(r"\b{}\b".format(variant), arm_title.lower())):
    #     # print("found", "Arm Title")
    #     variant_found_in.append(f"{variant} = Arm Title")
    #     variants_found_list.append(variant)
    #   if (re.findall(r"\b{}\b".format(variant), arm_description.lower())):
    #     # print("found", "Arm Description")
    #     variant_found_in.append(f"{variant} = Arm Description")
    #     variants_found_list.append(variant)
    #   if (re.findall(r"\b{}\b".format(variant), eligibility_criteria.lower())):
    #     # print("found", "Eligibility Criterial")
    #     variant_found_in.append(f"{variant} = Eligibility Criteria")
    #     variants_found_list.append(variant)
    #   if (re.findall(r"\b{}\b".format(variant), inclusion_criteria.lower())):
    #     # print("found", "Inclusion Criteria")
    #     variant_found_in.append(f"{variant} = Inclusion Criteria")
    #     variants_found_list.append(variant)
    #   if (re.findall(r"\b{}\b".format(variant), exclusion_criteria.lower())):
    #     # print("found", "Exclusion Criteria")
    #     variant_found_in.append(f"{variant} = Exclusion Criteria")
    #     variants_found_list.append(variant)
    #   if (re.findall(r"\b{}\b".format(variant), keywords.lower())):
    #     # print("found", "Keywords")
    #     variant_found_in.append(f"{variant} = Keywords")
    #     variants_found_list.append(variant)
    #   if (re.findall(r"\b{}\b".format(variant), primary_om_title.lower())):
    #     # print("found", "Primary OM Title")
    #     variant_found_in.append(f"{variant} = Primary OM Title")
    #     variants_found_list.append(variant)
    #   if (re.findall(r"\b{}\b".format(variant), primary_om_description.lower())):
    #     # print("found", "Primary OM Description")
    #     variant_found_in.append(f"{variant} = Primary OM Description")
    #     variants_found_list.append(variant)
    #   if (re.findall(r"\b{}\b".format(variant), secondary_om_title.lower())):
    #     # print("found", "Secondary OM Title")
    #     variant_found_in.append(f"{variant} = Secondary OM Title")
    #     variants_found_list.append(variant)
    #   if (re.findall(r"\b{}\b".format(variant), secondary_om_description.lower())):
    #     # print("found", "Secondary OM Description")
    #     variant_found_in.append(f"{variant} = Secondary OM Description")
    #     variants_found_list.append(variant)
    #   if (re.findall(r"\b{}\b".format(variant), other_om_title.lower())):
    #     # print("found", "Other OM Title")
    #     variant_found_in.append(f"{variant} = Other OM Title")
    #     variants_found_list.append(variant)
    #   if (re.findall(r"\b{}\b".format(variant), other_om_description.lower())):
    #     # print("found", "Other OM Description")
    #     variant_found_in.append(f"{variant} = Other OM Description")
    #     variants_found_list.append(variant)
    #   if (re.findall(r"\b{}\b".format(variant), mesh_terms.lower())):
    #     # print("found", "MeSH Terms")
    #     variant_found_in.append(f"{variant} = MeSH Terms")
    #     variants_found_list.append(variant)

    # variant_found_in_string = " | ".join(variant_found_in)
    # variant_found_in_list = variant_found_in_string.split(" | ")

    # variants_found_list = list(set(variants_found_list))
    # variants_found_string = " | ".join(variants_found_list)


    # print("Variant found in completed")

    # -------------------------------------- For assigning rank for the nct id --------------------------------------------

    rank = ""

    final_check_list = []
    if "" not in gene_found_in_list:
      # print(gene_found_in_list)
      for gene_value in gene_found_in_list:
        check_data = gene_value.split(" = ")[1]
        final_check_list.append(check_data)

      for column_value in final_check_list:

        if "Inclusion Criteria" in final_check_list or "Official Title" in final_check_list or "Cancer Type" in final_check_list or "Arm Title" in final_check_list or "Arm Description" in final_check_list or "Brief Title" in final_check_list or "Intervention Description" in final_check_list:
          rank = "1A"
        elif "Brief Summary" in final_check_list or "Detailed Description" in final_check_list or "Keywords" in final_check_list:
          rank = "1B"
        elif "Exclusion Criteria" in final_check_list:
          rank = "3"
        elif "Eligibility Criteria" in final_check_list:
          rank = "1C"
        elif "Primary OM Title" in final_check_list or "Primary OM Description" in final_check_list or "Secondary OM Title" in final_check_list or "Secondary OM Description" in final_check_list or "Other OM Title" in final_check_list or "Other OM Description" in final_check_list or "MeSH Terms" in final_check_list:
          rank = "2"
    # -------------------------------------- For assigning rank for the nct id --------------------------------------------

    # ---------------------------------------For assiging drugs and pathway --------------------------------------------------------
    
    # drug_check_list = []
    # drug_not_found_list = []
    # pathway_found_list = []

    # if "||" in intervention:
    #   drug_list = intervention.split("||")
    #   for drug in drug_list:
    #     drug = drug.strip()
    #     if drug.title() in target_pathway_df['DrugName'].values or drug.lower() in target_pathway_df['DrugName'].values or drug.upper() in target_pathway_df['DrugName'].values:
    #       result = target_pathway_df.loc[(target_pathway_df['DrugName'].str.lower() == drug.lower())]
    #       drug_match = result['Target'].to_string(index=False).strip()
    #       pathway_match = result['Pathway'].to_string(index=False).strip()
    #       drug_check_list.append(f"{drug} = {drug_match}")
    #       pathway_found_list.append(f"{4/1AX4XfWhVSjrkAeRzFOloPa-ofd141lniTyYBlRlHyn4crNVhJKtS2CtHICEdrug} = {pathway_match}")
    #     else:
    #       drug_not_found_list.append(drug)
    # elif "and" in intervention:
    #   drug_list = intervention.split("and")
    #   for drug in drug_list:
    #     drug = drug.strip()
    #     # print("2 executed")
    #     if drug.title() in target_pathway_df['DrugName'].values or drug.lower() in target_pathway_df['DrugName'].values or drug.upper() in target_pathway_df['DrugName'].values:
    #       # print("found2")
    #       result = target_pathway_df.loc[(target_pathway_df['DrugName'].str.lower() == drug.lower())]
    #       drug_match = result['Target'].to_string(index=False).strip()
    #       pathway_match = result['Pathway'].to_string(index=False).strip()
    #       drug_check_list.append(f"{drug} = {drug_match}")
    #       pathway_found_list.append(f"{drug} = {pathway_match}")
    #     else:
    #       drug_not_found_list.append(drug)
    # else:
    #   drug = intervention.strip()
    #   if drug.title() in target_pathway_df['DrugName'].values or dEGFR - completed in 14.0 minutes rug.lower() in target_pathway_df['DrugName'].values or drug.upper() in target_pathway_df['DrugName'].values:
    #     # print("found2")
    #     result = target_pathway_df.loc[(target_pathway_df['DrugName'].str.lower() == drug.lower())]
    #     drug_match = result['Target'].to_string(index=False).strip()
    #     drug_check_list.append(f"{drug} = {drug_match}")
    #   else:
    #     drug_not_found_list.append(drug)'Cancer Type' 'Intervention


    # drug_check_string = " ; ".join(drug_check_list)
    # pathway_found_string = " ; ".join(pathway_found_list)
    # drug_not_found_string = " | ".join(drug_not_found_list)
    # #  -----------------------------------------------------------------------------------------------------------------------------------------

        
    trial_rows.append(nct_id)
    trial_rows.append(study_type)
    trial_rows.append(study_purpose)
    trial_rows.append(study_model)
    trial_rows.append(overall_status)
    trial_rows.append(phase)
    trial_rows.append(rank)
    trial_rows.append(gene_found_in_string)
    trial_rows.append(genes_found_string)
    # trial_rows.append(variant_found_in_string)
    # trial_rows.append(variants_found_string)
    trial_rows.append(brief_title)
    trial_rows.append(official_title)
    trial_rows.append(brief_summary)
    trial_rows.append(detailed_description)
    trial_rows.append(cancer_type)
    trial_rows.append(intervention)
    trial_rows.append(intervention_description)
    trial_rows.append(intervention_other_name)
    trial_rows.append(arm_title)
    trial_rows.append(arm_description)
    trial_rows.append(arm_group_intervention)
    trial_rows.append(eligibility_criteria)
    trial_rows.append(inclusion_criteria)
    trial_rows.append(exclusion_criteria)
    trial_rows.append(character_count)
    trial_rows.append(primary_om_title)
    trial_rows.append(primary_om_description)
    trial_rows.append(secondary_om_title)
    trial_rows.append(secondary_om_description)
    trial_rows.append(other_om_title)
    trial_rows.append(other_om_description)
    trial_rows.append(keywords)
    trial_rows.append(mesh_terms)
    trial_rows.append(pmid)
    trial_rows.append(url)
    
    # trial_rows.append(variant_found_in_string)
    # trial_rows.append(drug_check_string)
    # trial_rows.append(pathway_found_string)
    # trial_rows.append(drug_not_found_string)
    
    result_list.append(trial_rows)

  # result_list_df = pd.DataFrame(result_list, columns = ["NCT ID", "Study Type", "Recruitment Status", "Phase", "Brief Title", "Official Title", "Brief Summary", "Detailed Description", "Cancer Type", "Study Purpose", "Intervention", "Intervention Description", "Intervention Other Name", "Study Model", "Arm Title", "Arm Description", "Arm Group Intervention", "Eligibility Criteria", "Inclusion Criteria", "Exclusion Criteria", "Keywords", "PMID", "URL", "Primary OM Title", "Primary OM Description", "Secondary OM Title", "Secondary OM Description", "Other OM Title", "Other OM Description", "MeSH Terms", "Character Count", "Gene Found In", "Rank", "Drug Match", "Pathway Match", "Drug Not Found"])
  result_list_df = pd.DataFrame(result_list, columns = ['NCT ID', 'Study Type', 'Study Purpose', 'Study Model', 'Recruitment Status', 'Phase', 'Level', 'Gene Found In', 'Gene Aliases Found', 'Brief Title', 'Official Title', 'Brief Summary', 'Detailed Description', 'Cancer Type', 'Intervention', 'Intervention Description', 'Intervention Other Name', 'Arm Title', 'Arm Description', 'Arm Group Intervention', 'Eligibility Criteria', 'Inclusion Criteria', 'Exclusion Criteria', 'Character Count', 'Primary OM Title', 'Primary OM Description', 'Secondary OM Title', 'Secondary OM Description', 'Other OM Title', 'Other OM Description', 'Keywords', 'MeSH Terms', 'PMID', 'URL'])
  result_list_df = result_list_df[result_list_df['Gene Found In'] != ""]
  return result_list_df
    

In [85]:
def variant_exact_match(sheet_df, gene):
  #get Variant Names
  variant_sh = gc.open_by_key("1_hKejRNMKYEf4jPQYSvzJZZUHIm-eNDDcuyc8LB6L5M")
  variant_sheet = variant_sh.worksheet('DummyData')
  variant_rows = variant_sheet.get_all_values()

  #converting to dataframe
  variant_df = pd.DataFrame(variant_rows[1:], columns=variant_rows[0])

  # drop rows which is having none and unknown
  variant_df = variant_df[variant_df['Impact'] != "none"]
  variant_df = variant_df[variant_df['Impact'] != "unknown"]

  data = variant_df.loc[(variant_df.Gene == gene)]
  variant_names = data['Variant'].to_numpy().tolist()
  
  result_list = [] 
  for index in sheet_df.index: # nct = 1, 2
  
    
    # variant_found_in = []
    # variants_found_list = []

    data_df = sheet_df.loc[index]
  
    
    nct_id = data_df['NCT ID']
    study_type = data_df['Study Type']
    overall_status = data_df['Recruitment Status']
    phase = data_df['Phase']
    brief_title = data_df['Brief Title']
    official_title = data_df['Official Title']
    brief_summary = data_df['Brief Summary']
    detailed_description = data_df['Detailed Description']
    cancer_type = data_df['Cancer Type']
    study_purpose = data_df['Study Purpose']
    intervention = data_df['Intervention']
    intervention_description = data_df['Intervention Description']
    intervention_other_name = data_df['Intervention Other Name']
    study_model = data_df['Study Model']
    arm_title = data_df['Arm Title']
    arm_description = data_df['Arm Description']
    arm_group_intervention = data_df['Arm Group Intervention']
    eligibility_criteria = data_df['Eligibility Criteria']
    inclusion_criteria = data_df['Inclusion Criteria']
    exclusion_criteria = data_df['Exclusion Criteria']
    keywords = data_df['Keywords']
    pmid = data_df['PMID']
    url = data_df['URL']
    primary_om_title = data_df['Primary OM Title']
    primary_om_description = data_df['Primary OM Description']
    secondary_om_title = data_df['Secondary OM Title']
    secondary_om_description = data_df['Secondary OM Description']
    other_om_title = data_df['Other OM Title']
    other_om_description = data_df['Other OM Description']
    mesh_terms = data_df['MeSH Terms']
    character_count = data_df['Character Count']
    gene_found_in = data_df['Gene Found In']
    gene_aliases_found = data_df['Gene Aliases Found']
    rank = data_df['Level']

    count = 0

    for variant in variant_names: #a, b
      variant = variant.lower()
      trial_rows=[]
      variant_found_in = []
      variants_found_list = []
      if (re.findall(r"\b{}\b".format(variant), brief_title.lower())):
        # print("found", "Brief Title")
        variant_found_in.append(f"{variant} = Brief Title")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(variant), official_title.lower())):
        # print("found", "Official Title")
        variant_found_in.append(f"{variant} = Official Title")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(variant), brief_summary.lower())):
        # print("found", "Brief Summary")
        variant_found_in.append(f"{variant} = Brief Summary")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(variant), detailed_description.lower())):
        # print("found", "Detailed Description")
        variant_found_in.append(f"{variant} = Detailed Description")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(variant), cancer_type.lower())):
        # print("found", "Cancer Type")
        variant_found_in.append(f"{variant} = Cancer Type")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(variant), intervention_description.lower())):
        # print("found", "Intervention Description")
        variant_found_in.append(f"{variant} = Intervention Description")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(variant), arm_title.lower())):
        # print("found", "Arm Title")
        variant_found_in.append(f"{variant} = Arm Title")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(variant), arm_description.lower())):
        # print("found", "Arm Description")
        variant_found_in.append(f"{variant} = Arm Description")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(variant), eligibility_criteria.lower())):
        # print("found", "Eligibility Criterial")
        variant_found_in.append(f"{variant} = Eligibility Criteria")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(variant), inclusion_criteria.lower())):
        # print("found", "Inclusion Criteria")
        variant_found_in.append(f"{variant} = Inclusion Criteria")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(variant), exclusion_criteria.lower())):
        # print("found", "Exclusion Criteria")
        variant_found_in.append(f"{variant} = Exclusion Criteria")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(variant), keywords.lower())):
        # print("found", "Keywords")
        variant_found_in.append(f"{variant} = Keywords")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(variant), primary_om_title.lower())):
        # print("found", "Primary OM Title")
        variant_found_in.append(f"{variant} = Primary OM Title")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(variant), primary_om_description.lower())):
        # print("found", "Primary OM Description")
        variant_found_in.append(f"{variant} = Primary OM Description")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(variant), secondary_om_title.lower())):
        # print("found", "Secondary OM Title")
        variant_found_in.append(f"{variant} = Secondary OM Title")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(variant), secondary_om_description.lower())):
        # print("found", "Secondary OM Description")
        variant_found_in.append(f"{variant} = Secondary OM Description")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(variant), other_om_title.lower())):
        # print("found", "Other OM Title")
        variant_found_in.append(f"{variant} = Other OM Title")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(variant), other_om_description.lower())):
        # print("found", "Other OM Description")
        variant_found_in.append(f"{variant} = Other OM Description")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(variant), mesh_terms.lower())):
        # print("found", "MeSH Terms")
        variant_found_in.append(f"{variant} = MeSH Terms")
        variants_found_list.append(variant)

      variant_found_in_string = " | ".join(variant_found_in)
      variant_found_in_list = variant_found_in_string.split(" | ")

      variants_found_list = list(set(variants_found_list))
      variants_found_string = " | ".join(variants_found_list)

      if len(variants_found_list) != 0:
      
        trial_rows.append(nct_id)
        trial_rows.append(study_type)
        trial_rows.append(study_purpose)
        trial_rows.append(study_model)
        trial_rows.append(overall_status)
        trial_rows.append(phase)
        trial_rows.append(rank)
        trial_rows.append(gene_found_in)
        trial_rows.append(gene_aliases_found)
        trial_rows.append(variant_found_in_string)
        trial_rows.append(variants_found_string)
        trial_rows.append("Exact Match")
        trial_rows.append(brief_title)
        trial_rows.append(official_title)
        trial_rows.append(brief_summary)
        trial_rows.append(detailed_description)
        trial_rows.append(cancer_type)
        trial_rows.append(intervention)
        trial_rows.append(intervention_description)
        trial_rows.append(intervention_other_name)
        trial_rows.append(arm_title)
        trial_rows.append(arm_description)
        trial_rows.append(arm_group_intervention)
        trial_rows.append(eligibility_criteria)
        trial_rows.append(inclusion_criteria)
        trial_rows.append(exclusion_criteria)
        trial_rows.append(character_count)
        trial_rows.append(primary_om_title)
        trial_rows.append(primary_om_description)
        trial_rows.append(secondary_om_title)
        trial_rows.append(secondary_om_description)
        trial_rows.append(other_om_title)
        trial_rows.append(other_om_description)
        trial_rows.append(keywords)
        trial_rows.append(mesh_terms)
        trial_rows.append(pmid)
        trial_rows.append(url)
        result_list.append(trial_rows)

        count = count + 1
    
    if count == 0:

      trial_rows.append(nct_id)
      trial_rows.append(study_type)
      trial_rows.append(study_purpose)
      trial_rows.append(study_model)
      trial_rows.append(overall_status)
      trial_rows.append(phase)
      trial_rows.append(rank)
      trial_rows.append(gene_found_in)
      trial_rows.append(gene_aliases_found)
      trial_rows.append(variant_found_in_string)
      trial_rows.append(variants_found_string)
      trial_rows.append("VNF")
      trial_rows.append(brief_title)
      trial_rows.append(official_title)
      trial_rows.append(brief_summary)
      trial_rows.append(detailed_description)
      trial_rows.append(cancer_type)
      trial_rows.append(intervention)
      trial_rows.append(intervention_description)
      trial_rows.append(intervention_other_name)
      trial_rows.append(arm_title)
      trial_rows.append(arm_description)
      trial_rows.append(arm_group_intervention)
      trial_rows.append(eligibility_criteria)
      trial_rows.append(inclusion_criteria)
      trial_rows.append(exclusion_criteria)
      trial_rows.append(character_count)
      trial_rows.append(primary_om_title)
      trial_rows.append(primary_om_description)
      trial_rows.append(secondary_om_title)
      trial_rows.append(secondary_om_description)
      trial_rows.append(other_om_title)
      trial_rows.append(other_om_description)
      trial_rows.append(keywords)
      trial_rows.append(mesh_terms)
      trial_rows.append(pmid)
      trial_rows.append(url)
      result_list.append(trial_rows)

  
    
        
    # trial_rows.append(variant_found_in_string)
    # trial_rows.append(drug_check_string)
    # trial_rows.append(pathway_found_string)
    # trial_rows.append(drug_not_found_string)
    
    # result_list.append(trial_rowsresult_list.append(trial_rows))

  # result_list_df = pd.DataFrame(result_list, columns = ["NCT ID", "Study Type", "Recruitment Status", "Phase", "Brief Title", "Official Title", "Brief Summary", "Detailed Description", "Cancer Type", "Study Purpose", "Intervention", "Intervention Description", "Intervention Other Name", "Study Model", "Arm Title", "Arm Description", "Arm Group Intervention", "Eligibility Criteria", "Inclusion Criteria", "Exclusion Criteria", "Keywords", "PMID", "URL", "Primary OM Title", "Primary OM Description", "Secondary OM Title", "Secondary OM Description", "Other OM Title", "Other OM Description", "MeSH Terms", "Character Count", "Gene Found In", "Rank", "Drug Match", "Pathway Match", "Drug Not Found"])
  result_list_df = pd.DataFrame(result_list, columns = ['NCT ID', 'Study Type', 'Study Purpose', 'Study Model', 'Recruitment Status', 'Phase', 'Level', 'Gene Found In', 'Gene Aliases Found', 'Variant Found In', 'Variants Found', 'Variant Match Type', 'Brief Title', 'Official Title', 'Brief Summary', 'Detailed Description', 'Cancer Type', 'Intervention', 'Intervention Description', 'Intervention Other Name', 'Arm Title', 'Arm Description', 'Arm Group Intervention', 'Eligibility Criteria', 'Inclusion Criteria', 'Exclusion Criteria', 'Character Count', 'Primary OM Title', 'Primary OM Description', 'Secondary OM Title', 'Secondary OM Description', 'Other OM Title', 'Other OM Description', 'Keywords', 'MeSH Terms', 'PMID', 'URL'])
  # result_list_df = result_list_df[result_list_df['Gene Found In'] != ""]
  return result_list_df
    


In [86]:
def missense_codon_match(sheet_df, gene):
  #get Variant Names
  variant_sh = gc.open_by_key("1_hKejRNMKYEf4jPQYSvzJZZUHIm-eNDDcuyc8LB6L5M")
  variant_sheet = variant_sh.worksheet('DummyData')
  variant_rows = variant_sheet.get_all_values()

  variant_df = pd.DataFrame(variant_rows[1:], columns=variant_rows[0])

  variant_df = variant_df[variant_df['Impact'] == "missense"]

  data = variant_df.loc[(variant_df.Gene == gene)]


  variants = data['Variant'].to_numpy().tolist()
  codons = data['Codon'].to_numpy().tolist()

  i = 0
  variant_names = []
  for i in range(len(variants)):
    variant = f"{variants[i]}|{codons[i]}"
    variant_names.append(variant)
  
  result_list = [] 
  for index in sheet_df.index: # nct = 1, 2

    data_df = sheet_df.loc[index]
  
    nct_id = data_df['NCT ID']
    study_type = data_df['Study Type']
    overall_status = data_df['Recruitment Status']
    phase = data_df['Phase']
    brief_title = data_df['Brief Title'].lower()
    official_title = data_df['Official Title'].lower()
    brief_summary = data_df['Brief Summary'].lower()
    detailed_description = data_df['Detailed Description'].lower()
    cancer_type = data_df['Cancer Type'].lower()
    study_purpose = data_df['Study Purpose']
    intervention = data_df['Intervention']
    intervention_description = data_df['Intervention Description'].lower()
    intervention_other_name = data_df['Intervention Other Name']
    study_model = data_df['Study Model']
    arm_title = data_df['Arm Title'].lower()
    arm_description = data_df['Arm Description'].lower()
    arm_group_intervention = data_df['Arm Group Intervention']
    eligibility_criteria = data_df['Eligibility Criteria'].lower()
    inclusion_criteria = data_df['Inclusion Criteria'].lower()
    exclusion_criteria = data_df['Exclusion Criteria'].lower()
    keywords = data_df['Keywords'].lower()
    pmid = data_df['PMID']
    url = data_df['URL']
    primary_om_title = data_df['Primary OM Title'].lower()
    primary_om_description = data_df['Primary OM Description'].lower()
    secondary_om_title = data_df['Secondary OM Title'].lower()
    secondary_om_description = data_df['Secondary OM Description'].lower()
    other_om_title = data_df['Other OM Title'].lower()
    other_om_description = data_df['Other OM Description'].lower()
    mesh_terms = data_df['MeSH Terms'].lower()
    character_count = data_df['Character Count']
    gene_found_in = data_df['Gene Found In']
    gene_aliases_found = data_df['Gene Aliases Found']
    rank = data_df['Level']

    brief_title = brief_title.replace("codons", "codon")
    official_title = official_title.replace("codons", "codon")
    brief_summary = brief_summary.replace("codons", "codon")
    detailed_description = detailed_description.replace("codons", "codon")
    cancer_type = cancer_type.replace("codons", "codon")
    intervention_description = intervention_description.replace("codons", "codon")
    arm_title = arm_title.replace("codons", "codon")
    arm_description = arm_description.replace("codons", "codon")
    eligibility_criteria = eligibility_criteria.replace("codons", "codon")
    inclusion_criteria = inclusion_criteria.replace("codons", "codon")
    exclusion_criteria = exclusion_criteria.replace("codons", "codon")
    keywords = keywords.replace("codons", "codon")
    primary_om_title = primary_om_title.replace("codons", "codon")
    primary_om_description = primary_om_description.replace("codons", "codon")
    secondary_om_title = secondary_om_title.replace("codons", "codon")
    secondary_om_description = secondary_om_description.replace("codons", "codon")
    other_om_title = other_om_title.replace("codons", "codon")
    other_om_description = other_om_description.replace("codons", "codon")
    mesh_terms = mesh_terms.replace("codons", "codon")


    count = 0

    for variant in variant_names: #a, b
      variant, codon = variant.split("|")
      codon = f"codon {codon}"
      codon = codon.lower()
      trial_rows=[]
      variant_found_in = []
      variants_found_list = []
      if (re.findall(r"\b{}\b".format(codon), brief_title.lower())):
        # print("found", "Brief Title")
        variant_found_in.append(f"{codon} = Brief Title")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(codon), official_title.lower())):
        # print("found", "Official Title")
        variant_found_in.append(f"{codon} = Official Title")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(codon), brief_summary.lower())):
        # print("found", "Brief Summary")
        variant_found_in.append(f"{codon} = Brief Summary")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(codon), detailed_description.lower())):
        # print("found", "Detailed Description")
        variant_found_in.append(f"{codon} = Detailed Description")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(codon), cancer_type.lower())):
        # print("found", "Cancer Type")
        variant_found_in.append(f"{codon} = Cancer Type")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(codon), intervention_description.lower())):
        # print("found", "Intervention Description")
        variant_found_in.append(f"{codon} = Intervention Description")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(codon), arm_title.lower())):
        # print("found", "Arm Title")
        variant_found_in.append(f"{codon} = Arm Title")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(codon), arm_description.lower())):
        # print("found", "Arm Description")
        variant_found_in.append(f"{codon} = Arm Description")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(codon), eligibility_criteria.lower())):
        # print("found", "Eligibility Criterial")
        variant_found_in.append(f"{codon} = Eligibility Criteria")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(codon), inclusion_criteria.lower())):
        # print("found", "Inclusion Criteria")
        variant_found_in.append(f"{codon} = Inclusion Criteria")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(codon), exclusion_criteria.lower())):
        # print("found", "Exclusion Criteria")
        variant_found_in.append(f"{codon} = Exclusion Criteria")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(codon), keywords.lower())):
        # print("found", "Keywords")
        variant_found_in.append(f"{codon} = Keywords")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(codon), primary_om_title.lower())):
        # print("found", "Primary OM Title")
        variant_found_in.append(f"{codon} = Primary OM Title")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(codon), primary_om_description.lower())):
        # print("found", "Primary OM Description")
        variant_found_in.append(f"{codon} = Primary OM Description")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(codon), secondary_om_title.lower())):
        # print("found", "Secondary OM Title")
        variant_found_in.append(f"{codon} = Secondary OM Title")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(codon), secondary_om_description.lower())):
        # print("found", "Secondary OM Description")
        variant_found_in.append(f"{codon} = Secondary OM Description")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(codon), other_om_title.lower())):
        # print("found", "Other OM Title")
        variant_found_in.append(f"{codon} = Other OM Title")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(codon), other_om_description.lower())):
        # print("found", "Other OM Description")
        variant_found_in.append(f"{codon} = Other OM Description")
        variants_found_list.append(variant)
      if (re.findall(r"\b{}\b".format(codon), mesh_terms.lower())):
        # print("found", "MeSH Terms")
        variant_found_in.append(f"{codon} = MeSH Terms")
        variants_found_list.append(variant)

      variant_found_in_string = " | ".join(variant_found_in)
      variant_found_in_list = variant_found_in_string.split(" | ")

      variants_found_list = list(set(variants_found_list))
      variants_found_string = " | ".join(variants_found_list)

      if len(variants_found_list) != 0:
      
        trial_rows.append(nct_id)
        trial_rows.append(study_type)
        trial_rows.append(study_purpose)
        trial_rows.append(study_model)
        trial_rows.append(overall_status)
        trial_rows.append(phase)
        trial_rows.append(rank)
        trial_rows.append(gene_found_in)
        trial_rows.append(gene_aliases_found)
        trial_rows.append(variant_found_in_string)
        trial_rows.append(variants_found_string)
        trial_rows.append("Codon Match")
        trial_rows.append(brief_title)
        trial_rows.append(official_title)
        trial_rows.append(brief_summary)
        trial_rows.append(detailed_description)
        trial_rows.append(cancer_type)
        trial_rows.append(intervention)
        trial_rows.append(intervention_description)
        trial_rows.append(intervention_other_name)
        trial_rows.append(arm_title)
        trial_rows.append(arm_description)
        trial_rows.append(arm_group_intervention)
        trial_rows.append(eligibility_criteria)
        trial_rows.append(inclusion_criteria)
        trial_rows.append(exclusion_criteria)
        trial_rows.append(character_count)
        trial_rows.append(primary_om_title)
        trial_rows.append(primary_om_description)
        trial_rows.append(secondary_om_title)
        trial_rows.append(secondary_om_description)
        trial_rows.append(other_om_title)
        trial_rows.append(other_om_description)
        trial_rows.append(keywords)
        trial_rows.append(mesh_terms)
        trial_rows.append(pmid)
        trial_rows.append(url)
        result_list.append(trial_rows)

        count = count + 1
    
    if count == 0:

      trial_rows.append(nct_id)
      trial_rows.append(study_type)
      trial_rows.append(study_purpose)
      trial_rows.append(study_model)
      trial_rows.append(overall_status)
      trial_rows.append(phase)
      trial_rows.append(rank)
      trial_rows.append(gene_found_in)
      trial_rows.append(gene_aliases_found)
      trial_rows.append(variant_found_in_string)
      trial_rows.append(variants_found_string)
      trial_rows.append("VNF")
      trial_rows.append(brief_title)
      trial_rows.append(official_title)
      trial_rows.append(brief_summary)
      trial_rows.append(detailed_description)
      trial_rows.append(cancer_type)
      trial_rows.append(intervention)
      trial_rows.append(intervention_description)
      trial_rows.append(intervention_other_name)
      trial_rows.append(arm_title)
      trial_rows.append(arm_description)
      trial_rows.append(arm_group_intervention)
      trial_rows.append(eligibility_criteria)
      trial_rows.append(inclusion_criteria)
      trial_rows.append(exclusion_criteria)
      trial_rows.append(character_count)
      trial_rows.append(primary_om_title)
      trial_rows.append(primary_om_description)
      trial_rows.append(secondary_om_title)
      trial_rows.append(secondary_om_description)
      trial_rows.append(other_om_title)
      trial_rows.append(other_om_description)
      trial_rows.append(keywords)
      trial_rows.append(mesh_terms)
      trial_rows.append(pmid)
      trial_rows.append(url)
      result_list.append(trial_rows)
      
    # trial_rows.append(variant_found_in_string)
    # trial_rows.append(drug_check_string)
    # trial_rows.append(pathway_found_string)
    # trial_rows.append(drug_not_found_string)
    
    # result_list.append(trial_rowsresult_list.append(trial_rows))

  # result_list_df = pd.DataFrame(result_list, columns = ["NCT ID", "Study Type", "Recruitment Status", "Phase", "Brief Title", "Official Title", "Brief Summary", "Detailed Description", "Cancer Type", "Study Purpose", "Intervention", "Intervention Description", "Intervention Other Name", "Study Model", "Arm Title", "Arm Description", "Arm Group Intervention", "Eligibility Criteria", "Inclusion Criteria", "Exclusion Criteria", "Keywords", "PMID", "URL", "Primary OM Title", "Primary OM Description", "Secondary OM Title", "Secondary OM Description", "Other OM Title", "Other OM Description", "MeSH Terms", "Character Count", "Gene Found In", "Rank", "Drug Match", "Pathway Match", "Drug Not Found"])
  result_list_df = pd.DataFrame(result_list, columns = ['NCT ID', 'Study Type', 'Study Purpose', 'Study Model', 'Recruitment Status', 'Phase', 'Level', 'Gene Found In', 'Gene Aliases Found', 'Variant Found In', 'Variants Found', 'Variant Match Type', 'Brief Title', 'Official Title', 'Brief Summary', 'Detailed Description', 'Cancer Type', 'Intervention', 'Intervention Description', 'Intervention Other Name', 'Arm Title', 'Arm Description', 'Arm Group Intervention', 'Eligibility Criteria', 'Inclusion Criteria', 'Exclusion Criteria', 'Character Count', 'Primary OM Title', 'Primary OM Description', 'Secondary OM Title', 'Secondary OM Description', 'Other OM Title', 'Other OM Description', 'Keywords', 'MeSH Terms', 'PMID', 'URL'])
  # result_list_df = result_list_df[result_list_df['Gene Found In'] != ""]
  return result_list_df
    


In [87]:
def duplication_codon_match(sheet_df, gene):
  variant_sh = gc.open_by_key("1_hKejRNMKYEf4jPQYSvzJZZUHIm-eNDDcuyc8LB6L5M")
  variant_sheet = variant_sh.worksheet('DummyData')
  variant_rows = variant_sheet.get_all_values()

  variant_df = pd.DataFrame(variant_rows[1:], columns=variant_rows[0])

  variant_df = variant_df[variant_df['Impact'] == "duplication"]  
  data = variant_df.loc[(variant_df.Gene == gene)]
  variants = data['Variant'].to_numpy().tolist()
  
  if len(variants) != 0:
    variant_names = []
    for variant in variants:
      if "_" in variant:
        codon1, codon2 = variant.split("dup")[0].split("_")
        codon1 = codon1 + "dup"
        codon2 = codon2 + "dup"
        codon1 = codon1.split("dup")[0][1:]
        codon2 = codon2.split("dup")[0][1:]
        variant_name = f"{variant}|{codon1}|{codon2}"
        variant_names.append(variant_name)
      else:
        codon = variant.split("dup")[0][1:]
        variant_name = f"{variant}|{codon}"
        variant_names.append(variant_name)

      
    result_list = [] 
    for index in sheet_df.index: # nct = 1, 2

      data_df = sheet_df.loc[index]
    
      nct_id = data_df['NCT ID']
      study_type = data_df['Study Type']
      overall_status = data_df['Recruitment Status']
      phase = data_df['Phase']
      brief_title = data_df['Brief Title'].lower()
      official_title = data_df['Official Title'].lower()
      brief_summary = data_df['Brief Summary'].lower()
      detailed_description = data_df['Detailed Description'].lower()
      cancer_type = data_df['Cancer Type'].lower()
      study_purpose = data_df['Study Purpose']
      intervention = data_df['Intervention']
      intervention_description = data_df['Intervention Description'].lower()
      intervention_other_name = data_df['Intervention Other Name']
      study_model = data_df['Study Model']
      arm_title = data_df['Arm Title'].lower()
      arm_description = data_df['Arm Description'].lower()
      arm_group_intervention = data_df['Arm Group Intervention']
      eligibility_criteria = data_df['Eligibility Criteria'].lower()
      inclusion_criteria = data_df['Inclusion Criteria'].lower()
      exclusion_criteria = data_df['Exclusion Criteria'].lower()
      keywords = data_df['Keywords'].lower()
      pmid = data_df['PMID']
      url = data_df['URL']
      primary_om_title = data_df['Primary OM Title'].lower()
      primary_om_description = data_df['Primary OM Description'].lower()
      secondary_om_title = data_df['Secondary OM Title'].lower()
      secondary_om_description = data_df['Secondary OM Description'].lower()
      other_om_title = data_df['Other OM Title'].lower()
      other_om_description = data_df['Other OM Description'].lower()
      mesh_terms = data_df['MeSH Terms'].lower()
      character_count = data_df['Character Count']
      gene_found_in = data_df['Gene Found In']
      gene_aliases_found = data_df['Gene Aliases Found']
      rank = data_df['Level']

      brief_title = brief_title.replace("codons", "codon")
      official_title = official_title.replace("codons", "codon")
      brief_summary = brief_summary.replace("codons", "codon")
      detailed_description = detailed_description.replace("codons", "codon")
      cancer_type = cancer_type.replace("codons", "codon")
      intervention_description = intervention_description.replace("codons", "codon")
      arm_title = arm_title.replace("codons", "codon")
      arm_description = arm_description.replace("codons", "codon")
      eligibility_criteria = eligibility_criteria.replace("codons", "codon")
      inclusion_criteria = inclusion_criteria.replace("codons", "codon")
      exclusion_criteria = exclusion_criteria.replace("codons", "codon")
      keywords = keywords.replace("codons", "codon")
      primary_om_title = primary_om_title.replace("codons", "codon")
      primary_om_description = primary_om_description.replace("codons", "codon")
      secondary_om_title = secondary_om_title.replace("codons", "codon")
      secondary_om_description = secondary_om_description.replace("codons", "codon")
      other_om_title = other_om_title.replace("codons", "codon")
      other_om_description = other_om_description.replace("codons", "codon")
      mesh_terms = mesh_terms.replace("codons", "codon")

      count = 0

      for variant in variant_names: #a, b
        if "_" in variant:
          for codon in variant.split("|")[1:]:
            if (re.findall(r"\b{}\b".format(codon), brief_title.lower())):
              # print("found", "Brief Title")
              variant_found_in.append(f"{codon} = Brief Title")
              variants_found_list.append(variant)
            if (re.findall(r"\b{}\b".format(codon), official_title.lower())):
              # print("found", "Official Title")
              variant_found_in.append(f"{codon} = Official Title")
              variants_found_list.append(variant)
            if (re.findall(r"\b{}\b".format(codon), brief_summary.lower())):
              # print("found", "Brief Summary")
              variant_found_in.append(f"{codon} = Brief Summary")
              variants_found_list.append(variant)
            if (re.findall(r"\b{}\b".format(codon), detailed_description.lower())):
              # print("found", "Detailed Description")
              variant_found_in.append(f"{codon} = Detailed Description")
              variants_found_list.append(variant)
            if (re.findall(r"\b{}\b".format(codon), cancer_type.lower())):
              # print("found", "Cancer Type")
              variant_found_in.append(f"{codon} = Cancer Type")
              variants_found_list.append(variant)
            if (re.findall(r"\b{}\b".format(codon), intervention_description.lower())):
              # print("found", "Intervention Description")
              variant_found_in.append(f"{codon} = Intervention Description")
              variants_found_list.append(variant)
            if (re.findall(r"\b{}\b".format(codon), arm_title.lower())):
              # print("found", "Arm Title")
              variant_found_in.append(f"{codon} = Arm Title")
              variants_found_list.append(variant)
            if (re.findall(r"\b{}\b".format(codon), arm_description.lower())):
              # print("found", "Arm Description")
              variant_found_in.append(f"{codon} = Arm Description")
              variants_found_list.append(variant)
            if (re.findall(r"\b{}\b".format(codon), eligibility_criteria.lower())):
              # print("found", "Eligibility Criterial")
              variant_found_in.append(f"{codon} = Eligibility Criteria")
              variants_found_list.append(variant)
            if (re.findall(r"\b{}\b".format(codon), inclusion_criteria.lower())):
              # print("found", "Inclusion Criteria")
              variant_found_in.append(f"{codon} = Inclusion Criteria")
              variants_found_list.append(variant)
            if (re.findall(r"\b{}\b".format(codon), exclusion_criteria.lower())):
              # print("found", "Exclusion Criteria")
              variant_found_in.append(f"{codon} = Exclusion Criteria")
              variants_found_list.append(variant)
            if (re.findall(r"\b{}\b".format(codon), keywords.lower())):
              # print("found", "Keywords")
              variant_found_in.append(f"{codon} = Keywords")
              variants_found_list.append(variant)
            if (re.findall(r"\b{}\b".format(codon), primary_om_title.lower())):
              # print("found", "Primary OM Title")
              variant_found_in.append(f"{codon} = Primary OM Title")
              variants_found_list.append(variant)
            if (re.findall(r"\b{}\b".format(codon), primary_om_description.lower())):
              # print("found", "Primary OM Description")
              variant_found_in.append(f"{codon} = Primary OM Description")
              variants_found_list.append(variant)
            if (re.findall(r"\b{}\b".format(codon), secondary_om_title.lower())):
              # print("found", "Secondary OM Title")
              variant_found_in.append(f"{codon} = Secondary OM Title")
              variants_found_list.append(variant)
            if (re.findall(r"\b{}\b".format(codon), secondary_om_description.lower())):
              # print("found", "Secondary OM Description")
              variant_found_in.append(f"{codon} = Secondary OM Description")
              variants_found_list.append(variant)
            if (re.findall(r"\b{}\b".format(codon), other_om_title.lower())):
              # print("found", "Other OM Title")
              variant_found_in.append(f"{codon} = Other OM Title")
              variants_found_list.append(variant)
            if (re.findall(r"\b{}\b".format(codon), other_om_description.lower())):
              # print("found", "Other OM Description")
              variant_found_in.append(f"{codon} = Other OM Description")
              variants_found_list.append(variant)
            if (re.findall(r"\b{}\b".format(codon), mesh_terms.lower())):
              # print("found", "MeSH Terms")
              variant_found_in.append(f"{codon} = MeSH Terms")
              variants_found_list.append(variant)

            variant_found_in_string = " | ".join(variant_found_in)
            variant_found_in_list = variant_found_in_string.split(" | ")

            variants_found_list = list(set(variants_found_list))
            variants_found_string = " | ".join(variants_found_list)

            if len(variants_found_list) != 0:
            
              trial_rows.append(nct_id)
              trial_rows.append(study_type)
              trial_rows.append(study_purpose)
              trial_rows.append(study_model)
              trial_rows.append(overall_status)
              trial_rows.append(phase)
              trial_rows.append(rank)
              trial_rows.append(gene_found_in)
              trial_rows.append(gene_aliases_found)
              trial_rows.append(variant_found_in_string)
              trial_rows.append(variants_found_string)
              trial_rows.append("Codon Match")
              trial_rows.append(brief_title)
              trial_rows.append(official_title)
              trial_rows.append(brief_summary)
              trial_rows.append(detailed_description)
              trial_rows.append(cancer_type)
              trial_rows.append(intervention)
              trial_rows.append(intervention_description)
              trial_rows.append(intervention_other_name)
              trial_rows.append(arm_title)
              trial_rows.append(arm_description)
              trial_rows.append(arm_group_intervention)
              trial_rows.append(eligibility_criteria)
              trial_rows.append(inclusion_criteria)
              trial_rows.append(exclusion_criteria)
              trial_rows.append(character_count)
              trial_rows.append(primary_om_title)
              trial_rows.append(primary_om_description)
              trial_rows.append(secondary_om_title)
              trial_rows.append(secondary_om_description)
              trial_rows.append(other_om_title)
              trial_rows.append(other_om_description)
              trial_rows.append(keywords)
              trial_rows.append(mesh_terms)
              trial_rows.append(pmid)
              trial_rows.append(url)
              result_list.append(trial_rows)

              count = count + 1
      
        variant, codon = variant.split("|")
        codon = f"codon {codon}"
        codon = codon.lower()
        trial_rows=[]
        variant_found_in = []
        variants_found_list = []
        if (re.findall(r"\b{}\b".format(codon), brief_title.lower())):
          # print("found", "Brief Title")
          variant_found_in.append(f"{codon} = Brief Title")
          variants_found_list.append(variant)
        if (re.findall(r"\b{}\b".format(codon), official_title.lower())):
          # print("found", "Official Title")
          variant_found_in.append(f"{codon} = Official Title")
          variants_found_list.append(variant)
        if (re.findall(r"\b{}\b".format(codon), brief_summary.lower())):
          # print("found", "Brief Summary")
          variant_found_in.append(f"{codon} = Brief Summary")
          variants_found_list.append(variant)
        if (re.findall(r"\b{}\b".format(codon), detailed_description.lower())):
          # print("found", "Detailed Description")
          variant_found_in.append(f"{codon} = Detailed Description")
          variants_found_list.append(variant)
        if (re.findall(r"\b{}\b".format(codon), cancer_type.lower())):
          # print("found", "Cancer Type")
          variant_found_in.append(f"{codon} = Cancer Type")
          variants_found_list.append(variant)
        if (re.findall(r"\b{}\b".format(codon), intervention_description.lower())):
          # print("found", "Intervention Description")
          variant_found_in.append(f"{codon} = Intervention Description")
          variants_found_list.append(variant)
        if (re.findall(r"\b{}\b".format(codon), arm_title.lower())):
          # print("found", "Arm Title")
          variant_found_in.append(f"{codon} = Arm Title")
          variants_found_list.append(variant)
        if (re.findall(r"\b{}\b".format(codon), arm_description.lower())):
          # print("found", "Arm Description")
          variant_found_in.append(f"{codon} = Arm Description")
          variants_found_list.append(variant)
        if (re.findall(r"\b{}\b".format(codon), eligibility_criteria.lower())):
          # print("found", "Eligibility Criterial")
          variant_found_in.append(f"{codon} = Eligibility Criteria")
          variants_found_list.append(variant)
        if (re.findall(r"\b{}\b".format(codon), inclusion_criteria.lower())):
          # print("found", "Inclusion Criteria")
          variant_found_in.append(f"{codon} = Inclusion Criteria")
          variants_found_list.append(variant)
        if (re.findall(r"\b{}\b".format(codon), exclusion_criteria.lower())):
          # print("found", "Exclusion Criteria")
          variant_found_in.append(f"{codon} = Exclusion Criteria")
          variants_found_list.append(variant)
        if (re.findall(r"\b{}\b".format(codon), keywords.lower())):
          # print("found", "Keywords")
          variant_found_in.append(f"{codon} = Keywords")
          variants_found_list.append(variant)
        if (re.findall(r"\b{}\b".format(codon), primary_om_title.lower())):
          # print("found", "Primary OM Title")
          variant_found_in.append(f"{codon} = Primary OM Title")
          variants_found_list.append(variant)
        if (re.findall(r"\b{}\b".format(codon), primary_om_description.lower())):
          # print("found", "Primary OM Description")
          variant_found_in.append(f"{codon} = Primary OM Description")
          variants_found_list.append(variant)
        if (re.findall(r"\b{}\b".format(codon), secondary_om_title.lower())):
          # print("found", "Secondary OM Title")
          variant_found_in.append(f"{codon} = Secondary OM Title")
          variants_found_list.append(variant)
        if (re.findall(r"\b{}\b".format(codon), secondary_om_description.lower())):
          # print("found", "Secondary OM Description")
          variant_found_in.append(f"{codon} = Secondary OM Description")
          variants_found_list.append(variant)
        if (re.findall(r"\b{}\b".format(codon), other_om_title.lower())):
          # print("found", "Other OM Title")
          variant_found_in.append(f"{codon} = Other OM Title")
          variants_found_list.append(variant)
        if (re.findall(r"\b{}\b".format(codon), other_om_description.lower())):
          # print("found", "Other OM Description")
          variant_found_in.append(f"{codon} = Other OM Description")
          variants_found_list.append(variant)
        if (re.findall(r"\b{}\b".format(codon), mesh_terms.lower())):
          # print("found", "MeSH Terms")
          variant_found_in.append(f"{codon} = MeSH Terms")
          variants_found_list.append(variant)

        variant_found_in_string = " | ".join(variant_found_in)
        variant_found_in_list = variant_found_in_string.split(" | ")

        variants_found_list = list(set(variants_found_list))
        variants_found_string = " | ".join(variants_found_list)

        if len(variants_found_list) != 0:
        
          trial_rows.append(nct_id)
          trial_rows.append(study_type)
          trial_rows.append(study_purpose)
          trial_rows.append(study_model)
          trial_rows.append(overall_status)
          trial_rows.append(phase)
          trial_rows.append(rank)
          trial_rows.append(gene_found_in)
          trial_rows.append(gene_aliases_found)
          trial_rows.append(variant_found_in_string)
          trial_rows.append(variants_found_string)
          trial_rows.append("Codon Match")
          trial_rows.append(brief_title)
          trial_rows.append(official_title)
          trial_rows.append(brief_summary)
          trial_rows.append(detailed_description)
          trial_rows.append(cancer_type)
          trial_rows.append(intervention)
          trial_rows.append(intervention_description)
          trial_rows.append(intervention_other_name)
          trial_rows.append(arm_title)
          trial_rows.append(arm_description)
          trial_rows.append(arm_group_intervention)
          trial_rows.append(eligibility_criteria)
          trial_rows.append(inclusion_criteria)
          trial_rows.append(exclusion_criteria)
          trial_rows.append(character_count)
          trial_rows.append(primary_om_title)
          trial_rows.append(primary_om_description)
          trial_rows.append(secondary_om_title)
          trial_rows.append(secondary_om_description)
          trial_rows.append(other_om_title)
          trial_rows.append(other_om_description)
          trial_rows.append(keywords)
          trial_rows.append(mesh_terms)
          trial_rows.append(pmid)
          trial_rows.append(url)
          result_list.append(trial_rows)

          count = count + 1
      
      if count == 0:

        trial_rows.append(nct_id)
        trial_rows.append(study_type)
        trial_rows.append(study_purpose)
        trial_rows.append(study_model)
        trial_rows.append(overall_status)
        trial_rows.append(phase)
        trial_rows.append(rank)
        trial_rows.append(gene_found_in)
        trial_rows.append(gene_aliases_found)
        trial_rows.append(variant_found_in_string)
        trial_rows.append(variants_found_string)
        trial_rows.append("VNF")
        trial_rows.append(brief_title)
        trial_rows.append(official_title)
        trial_rows.append(brief_summary)
        trial_rows.append(detailed_description)
        trial_rows.append(cancer_type)
        trial_rows.append(intervention)
        trial_rows.append(intervention_description)
        trial_rows.append(intervention_other_name)
        trial_rows.append(arm_title)
        trial_rows.append(arm_description)
        trial_rows.append(arm_group_intervention)
        trial_rows.append(eligibility_criteria)
        trial_rows.append(inclusion_criteria)
        trial_rows.append(exclusion_criteria)
        trial_rows.append(character_count)
        trial_rows.append(primary_om_title)
        trial_rows.append(primary_om_description)
        trial_rows.append(secondary_om_title)
        trial_rows.append(secondary_om_description)
        trial_rows.append(other_om_title)
        trial_rows.append(other_om_description)
        trial_rows.append(keywords)
        trial_rows.append(mesh_terms)
        trial_rows.append(pmid)
        trial_rows.append(url)
        result_list.append(trial_rows)
        
      # trial_rows.append(variant_found_in_string)
      # trial_rows.append(drug_check_string)
      # trial_rows.append(pathway_found_string)
      # trial_rows.append(drug_not_found_string)
      
      # result_list.append(trial_rowsresult_list.append(trial_rows))

    # result_list_df = pd.DataFrame(result_list, columns = ["NCT ID", "Study Type", "Recruitment Status", "Phase", "Brief Title", "Official Title", "Brief Summary", "Detailed Description", "Cancer Type", "Study Purpose", "Intervention", "Intervention Description", "Intervention Other Name", "Study Model", "Arm Title", "Arm Description", "Arm Group Intervention", "Eligibility Criteria", "Inclusion Criteria", "Exclusion Criteria", "Keywords", "PMID", "URL", "Primary OM Title", "Primary OM Description", "Secondary OM Title", "Secondary OM Description", "Other OM Title", "Other OM Description", "MeSH Terms", "Character Count", "Gene Found In", "Rank", "Drug Match", "Pathway Match", "Drug Not Found"])
    result_list_df = pd.DataFrame(result_list, columns = ['NCT ID', 'Study Type', 'Study Purpose', 'Study Model', 'Recruitment Status', 'Phase', 'Level', 'Gene Found In', 'Gene Aliases Found', 'Variant Found In', 'Variants Found', 'Variant Match Type', 'Brief Title', 'Official Title', 'Brief Summary', 'Detailed Description', 'Cancer Type', 'Intervention', 'Intervention Description', 'Intervention Other Name', 'Arm Title', 'Arm Description', 'Arm Group Intervention', 'Eligibility Criteria', 'Inclusion Criteria', 'Exclusion Criteria', 'Character Count', 'Primary OM Title', 'Primary OM Description', 'Secondary OM Title', 'Secondary OM Description', 'Other OM Title', 'Other OM Description', 'Keywords', 'MeSH Terms', 'PMID', 'URL'])
    # result_list_df = result_list_df[result_list_df['Gene Found In'] != ""]
    return result_list_df
  else:
    return sheet_df


In [ ]:
# def rowsplit(indata, col, sep):
#   s = indata.assign(variant_found=indata[col].str.split(
#         sep)).explode('Variants Found')
  
#   indata.drop(columns=[col], inplace=True)
#   # print(indata)
#   i = s.index.get_level_values(0)
#   # print(i)
#   indata = indata.loc[i].copy()
#   # print(indata)
#   indata["Variants Found"] = s['Variants Found']
#   # print(indata)
#   return indata


In [88]:
def generate_report(result_df, gene):
  final_report_list = []
  df_1a = result_df[result_df['Level'] == "1A"]
  df_1b = result_df[result_df['Level'] == "1B"]
  df_1c = result_df[result_df['Level'] == "1C"]

  report_df = pd.concat([df_1a, df_1b, df_1c])

  for index in report_df.index:
    trial_rows=[]
    data_df = report_df.loc[index]

    nct_id = data_df['NCT ID']
    phase = data_df['Phase']
    official_title = data_df['Official Title']
    intervention = data_df['Intervention']
    level = data_df['Level']
    cancer_type = data_df['Cancer Type']
    study_type = data_df['Study Type']
    pmid = data_df['PMID']
    url = data_df['URL']

    concatenated_string = f"{nct_id} | {phase} | {official_title}"

    trial_rows.append(gene)
    trial_rows.append(concatenated_string)
    trial_rows.append(intervention)
    trial_rows.append(level)
    trial_rows.append(nct_id)
    trial_rows.append(cancer_type)
    trial_rows.append(pmid)
    trial_rows.append(study_type)
    trial_rows.append(url)
    
    final_report_list.append(trial_rows)
  
  final_report_df = pd.DataFrame(final_report_list, columns = ['Gene', 'Concatenated String', 'Intervention', 'Level', 'NCT ID', 'Cancer Type', 'PMID', 'Study Type', 'URL'])
  # result_list_df = result_list_df[result_list_df['Gene Found In'] != ""]
  
  return final_report_df


In [89]:
def update_sheet(result_df, sheet_name, data_to_write, headers):
  today = date.today()
  sh = gc.open(f"{gene}-exactVariantMatch-{today}-try8")
  worksheet = sh.worksheet(sheet_name)
  print("started")
  try:
    worksheet.append_rows(data_to_write, value_input_option='USER_ENTERED', insert_data_option=None, table_range=None, include_values_in_response=False)
  except gspread.exceptions.APIError:
    worksheet = sh.add_worksheet(title=f"sheet_name"+1, rows="10000", col="50")
    worksheet.update("A1",headers,value_input_option="USER_ENTERED")
    worksheet.append_rows(data_to_write, value_input_option='USER_ENTERED', insert_data_option=None, table_range=None, include_values_in_response=False)
  print("write completed")
  worksheet.format("1:100000", {
  
    "horizontalAlignment": "LEFT",
    "verticalAlignment": "TOP",
    "wrapStrategy": "CLIP",
    
  })

  set_row_heights(worksheet, [ ('1:100000', 21), ('101:', 21) ])
  set_column_width(worksheet, 'A:AL', 200)

In [90]:

def update_spreadsheet(result_df, gene, sheet_name):
  today = date.today()
  sh = gc.open(f"{gene}-exactVariantMatch-{today}-try8")
  worksheet = sh.add_worksheet(title=sheet_name, rows="10000", cols="38")

  data_list = result_df.to_numpy().tolist()
  headers = result_df.columns.tolist()
  headers = [headers]
  worksheet.update("A1",headers,value_input_option="USER_ENTERED")

  end_limit = result_df.shape[0]
  print("completed")

  min_value = 0
  max_value = 999

  while max_value <= end_limit:
    data_to_write = data_list[min_value:max_value+1]
    update_sheet(result_df, sheet_name, data_to_write, headers)

    min_value += 1000
    max_value += 1000
  if max_value != end_limit:
    min_value = min_value
    max_value = end_limit
    data_to_write = data_list[min_value:max_value+1]
    update_sheet(result_df, sheet_name, data_to_write, headers)

In [38]:
interventional_trial_ss = gc.open_by_key("1cjT64UTRr8IDNATjeufEoebQN2k_vyCJU3eQkfbXYTo")

final_list = []
sheet_names = interventional_trial_ss.worksheets()
for sheet in sheet_names:
  
  rows = sheet.get_all_values()
  df = pd.DataFrame.from_records(rows[1:], columns=rows[0])
  final_list.append(df)

In [39]:
combined_df1 = pd.concat(final_list)
combined_df1.shape

(80316, 31)

In [40]:
observational_trial_ss = gc.open_by_key("1MlFhdqcwAa7Al1OPoe_3VnQ2kqQZ5-baG1-SWTW94xM")

final_list = []
sheet_names = observational_trial_ss.worksheets()
for sheet in sheet_names:
  if sheet.title != "Sheet1":
    rows = sheet.get_all_values()
    df = pd.DataFrame.from_records(rows[1:], columns=rows[0])
    final_list.append(df)

In [41]:
combined_df2 = pd.concat(final_list)
combined_df2.shape

(19147, 31)

In [42]:

expanded_trial_ss = gc.open_by_key("1hVXkqX_GBagL_PNTaqP-oFZ-4KaSUp5FwQBgXSzzvvA")

final_list = []
sheet_names = expanded_trial_ss.worksheets()
for sheet in sheet_names:
  if sheet.title != "Sheet1":
    rows = sheet.get_all_values()
    df = pd.DataFrame.from_records(rows[1:], columns=rows[0])
    final_list.append(df)

In [43]:
combined_df3 = pd.concat(final_list)
combined_df3.shape

(367, 31)

In [91]:
final_combined_df = pd.concat([combined_df1, combined_df2, combined_df3])
final_combined_df.set_index('NCT ID', inplace=True)
final_combined_df.reset_index(level=0, inplace=True)
final_combined_df.shape

(99830, 31)

In [ ]:
cancer_found_df = find_cancer_nct_id(final_combined_df)

In [ ]:
curate_genes_ss = gc.open_by_key("1jE7-Z9CGln7drOUnyFXO4cBwvu71I-SKKdaIL0xPvO8")
curate_genes_ws = curate_genes_ss.worksheet("InputGeneList")
rows = curate_genes_ws.get_all_values()
curate_genes_df = pd.DataFrame(rows[1:], columns=rows[0])
genes_list = curate_genes_df['exact match'].to_numpy().tolist()
genes_list

In [92]:
import time

In [63]:
genes_list = ["KRAS"]

for gene in genes_list:

  start = time.time()
  # result_list_df = pd.DataFrame(result_list, columns = ['NCT ID', 'Study Type', 'Study Purpose', 'Study Model', 'Recruitment Status', 'Phase', 'Level', 'Gene Found In', 'Gene Aliases Found', 'Variant Found In', 'Variants Found', 'Brief Title', 'Official Title', 'Brief Summary', 'Detailed Description', 'Cancer Type', 'Intervention', 'Intervention Description', 'Intervention Other Name', 'Arm Title', 'Arm Description', 'Arm Group Intervention', 'Eligibility Criteria', 'Inclusion Criteria', 'Exclusion Criteria', 'Character Count', 'Primary OM Title', 'Primary OM Description', 'Secondary OM Title', 'Secondary OM Description', 'Other OM Title', 'Other OM Description', 'Keywords', 'MeSH Terms', 'PMID', 'URL'])
  result_df = pd.DataFrame(columns = ['NCT ID', 'Study Type', 'Study Purpose', 'Study Model', 'Recruitment Status', 'Phase', 'Level', 'Gene Found In', 'Gene Aliases Found', 'Variant Match Type', 'Brief Title', 'Official Title', 'Brief Summary', 'Detailed Description', 'Cancer Type', 'Intervention', 'Intervention Description', 'Intervention Other Name', 'Arm Title', 'Arm Description', 'Arm Group Intervention', 'Eligibility Criteria', 'Inclusion Criteria', 'Exclusion Criteria', 'Character Count', 'Primary OM Title', 'Primary OM Description', 'Secondary OM Title', 'Secondary OM Description', 'Other OM Title', 'Other OM Description', 'Keywords', 'MeSH Terms', 'PMID', 'URL'])
  variant_result_df = pd.DataFrame(columns = ['NCT ID', 'Study Type', 'Study Purpose', 'Study Model', 'Recruitment Status', 'Phase', 'Level', 'Gene Found In', 'Gene Aliases Found', 'Variant Found In', 'Variants Found', 'Variant Match Type', 'Brief Title', 'Official Title', 'Brief Summary', 'Detailed Description', 'Cancer Type', 'Intervention', 'Intervention Description', 'Intervention Other Name', 'Arm Title', 'Arm Description', 'Arm Group Intervention', 'Eligibility Criteria', 'Inclusion Criteria', 'Exclusion Criteria', 'Character Count', 'Primary OM Title', 'Primary OM Description', 'Secondary OM Title', 'Secondary OM Description', 'Other OM Title', 'Other OM Description', 'Keywords', 'MeSH Terms', 'PMID', 'URL'])

  final_combined_df = final_combined_df[final_combined_df['Recruitment Status'] != "Terminated"]
  final_combined_df = final_combined_df[final_combined_df['Recruitment Status'] != "Suspended"]
  final_combined_df = final_combined_df[final_combined_df['Recruitment Status'] != "Unknown status"]
  final_combined_df = final_combined_df[final_combined_df['Recruitment Status'] != "Withdrawn"]

  #get gene names
  gene_names = get_gene_aliases_from_master_genes(gene)
  gene_names.append(gene)
  gene_names = list(set(gene_names))
  gene_alias_used = " | ".join(gene_names)

  final_df = filter_gene_found_in(sheet_df=final_combined_df, gene_names=gene_names)
  result_df = result_df.append(final_df)
  
  variant_df = variant_exact_match(sheet_df=result_df, gene=gene)
  variant_result_df1 = variant_result_df.append(variant_df)

  variant_exact_df = variant_df[variant_df['Variant Match Type'] == "Exact Match"]
  variant_vnf_df = variant_df[variant_df['Variant Match Type'] == "VNF"]

  variant_df = missense_codon_match(sheet_df=variant_vnf_df, gene=gene)
  variant_result_df2 = variant_result_df.append(variant_df)

  variant_codon_df = variant_df[variant_df['Variant Match Type'] == "Codon Match"]
  variant_vnf_df = variant_df[variant_df['Variant Match Type'] == "VNF"]

  variant_df = deletion_codon_match(sheet_df=variant_vnf_df, gene=gene)
  variant_result_df2 = variant_result_df.append(variant_df)

  result_df = pd.concat([variant_exact_df, variant_codon_df])

  total_ids = result_df.shape[0]
  level_1a_count = result_df[result_df['Level'] == "1A"].shape[0]
  level_1b_count = result_df[result_df['Level'] == "1B"].shape[0]
  level_1c_count = result_df[result_df['Level'] == "1C"].shape[0]
  level_2_count = result_df[result_df['Level'] == "2"].shape[0]
  level_3_count = result_df[result_df['Level'] == "3"].shape[0]

  gene_alias_found_list = result_df['Gene Aliases Found'].to_numpy().tolist()
  gene_alias_found_list = list(set(gene_alias_found_list))
  
  gene_alias_found_in_trial_list = []
  for alias in gene_alias_found_list:
    if "|" in alias:
      data = alias.split(" | ")
      for gene_value in data:
        gene_alias_found_in_trial_list.append(gene_value)
    else:
      gene_alias_found_in_trial_list.append(alias)

  gene_alias_found_in_trial_list = list(set(gene_alias_found_in_trial_list))

  gene_alias_found_in_trial_string = " | ".join(gene_alias_found_in_trial_list)
  today = date.today()
  # print(today)

  new_ss = gc.create(f"{gene}-exactVariantMatch-{today}-try7", "17x4-h4WPSOPPwdCdX9x595xBh6NknAXr")
  ws = new_ss.sheet1
  ws.update_title("Overview")
  
  ws.update(f"B3", "Gene Name")
  ws.update(f"C3", gene)
  ws.update(f"B4", "Gene Aliases Used to retrieve trials")
  ws.update(f"C4", gene_alias_used)
  ws.update(f"B5", "Gene Aliases found in trials")
  ws.update(f"C5", gene_alias_found_in_trial_string)
  ws.update(f"B6", "Total Count")
  ws.update(f"C6", total_ids)

  update_spreadsheet(result_df, gene, "Combined-Result")

  study_list = ['Interventional', 'Observational', 'Expanded Access', 'Patient Registry']
  level_list = ['1A', '1B', '1C', '2', '3']

  count = 8
  for study in study_list:
    df = result_df[result_df['Study Type'] == study]
    total_value = df.shape[0]
    ws.update(f"B{count}", f"{study} Studies")
    ws.update(f"C{count}", total_value)

    update_spreadsheet(df, gene, study)
    count = count+1

  count = 13
  for level in level_list:
    df = result_df[result_df['Level'] == level]
    total_value = df.shape[0]
    ws.update(f"B{count}", f"Level {level} trial")
    ws.update(f"C{count}", total_value)
    update_spreadsheet(df, gene, f"level {level}")
    count = count+1
  
  report_df = generate_report(result_df, gene)
  update_spreadsheet(report_df, gene, "Report")

  ws.format("1:10000", {
  
    "horizontalAlignment": "LEFT",
    "verticalAlignment": "TOP",
    "wrapStrategy": "WRAP",
    
  })

  
  set_column_width(ws, 'A:AL', 250)

  end = time.time()
  time_taken = (end - start) // 60
  # print(f"{gene}  completed in {time_taken} minutes ")
  print(gene, "\t completed in ", time_taken, " minutes","\t",total_ids, "\t 1A: ", level_1a_count, "\t 1B: ", level_1b_count, "\t 1C: ", level_1c_count, "\t 2: ", level_2_count, "\t 3: ", level_3_count, "\t GeneAlias: ", gene_alias_found_in_trial_string)

TypeError: ignored

In [48]:
result_df = pd.concat([variant_exact_df, variant_result_df2])

total_ids = result_df.shape[0]
level_1a_count = result_df[result_df['Level'] == "1A"].shape[0]
level_1b_count = result_df[result_df['Level'] == "1B"].shape[0]
level_1c_count = result_df[result_df['Level'] == "1C"].shape[0]
level_2_count = result_df[result_df['Level'] == "2"].shape[0]
level_3_count = result_df[result_df['Level'] == "3"].shape[0]

gene_alias_found_list = result_df['Gene Aliases Found'].to_numpy().tolist()
gene_alias_found_list = list(set(gene_alias_found_list))

gene_alias_found_in_trial_list = []
for alias in gene_alias_found_list:
  if "|" in alias:
    data = alias.split(" | ")
    for gene_value in data:
      gene_alias_found_in_trial_list.append(gene_value)
  else:
    gene_alias_found_in_trial_list.append(alias)

gene_alias_found_in_trial_list = list(set(gene_alias_found_in_trial_list))

gene_alias_found_in_trial_string = " | ".join(gene_alias_found_in_trial_list)
today = date.today()
# print(today)

new_ss = gc.create(f"{gene}-exactVariantMatch-{today}-try6", "17x4-h4WPSOPPwdCdX9x595xBh6NknAXr")
ws = new_ss.sheet1
ws.update_title("Overview")

ws.update(f"B3", "Gene Name")
ws.update(f"C3", gene)
ws.update(f"B4", "Gene Aliases Used to retrieve trials")
ws.update(f"C4", gene_alias_used)
ws.update(f"B5", "Gene Aliases found in trials")
ws.update(f"C5", gene_alias_found_in_trial_string)
ws.update(f"B6", "Total Count")
ws.update(f"C6", total_ids)

update_spreadsheet(result_df, gene, "Combined-Result")

study_list = ['Interventional', 'Observational', 'Expanded Access', 'Patient Registry']
level_list = ['1A', '1B', '1C', '2', '3']

count = 8
for study in study_list:
  df = result_df[result_df['Study Type'] == study]
  total_value = df.shape[0]
  ws.update(f"B{count}", f"{study} Studies")
  ws.update(f"C{count}", total_value)

  update_spreadsheet(df, gene, study)
  count = count+1

count = 13
for level in level_list:
  df = result_df[result_df['Level'] == level]
  total_value = df.shape[0]
  ws.update(f"B{count}", f"Level {level} trial")
  ws.update(f"C{count}", total_value)
  update_spreadsheet(df, gene, f"level {level}")
  count = count+1

report_df = generate_report(result_df, gene)
update_spreadsheet(report_df, gene, "Report")

ws.format("1:10000", {

  "horizontalAlignment": "LEFT",
  "verticalAlignment": "TOP",
  "wrapStrategy": "WRAP",
  
})


set_column_width(ws, 'A:AL', 250)

end = time.time()
time_taken = (end - start) // 60
# print(f"{gene}  completed in {time_taken} minutes ")
print(gene, "\t completed in ", time_taken, " minutes","\t",total_ids, "\t 1A: ", level_1a_count, "\t 1B: ", level_1b_count, "\t 1C: ", level_1c_count, "\t 2: ", level_2_count, "\t 3: ", level_3_count, "\t GeneAlias: ", gene_alias_found_in_trial_string)

TypeError: ignored

In [17]:
data = {"Test1": ['Hi How are you', 'this is a Codons Check', 'this is a codons check'], "Test2": ['This is a codons check', 'Codon check issue', "no value"]}
df = pd.DataFrame(data)
df

,Test1,Test2
0,Hi How are you,This is a codons check
1,this is a Codons Check,Codon check issue
2,this is a codons check,no value


In [12]:
df = df.replace(to_replace="codons", value="codon")
df

,Test1,Test2
0,Hi How are you,This is a codons check
1,this is a Codons Check,Codon check issue
2,this is a codon check,no value


In [22]:
for index in df.index:
  data_df = df.loc[index]
  test1 = data_df['Test1']
print(test1)
final = test1.replace("dfgdgfdg", "codon")
print(final)


this is a codons check
this is a codons check


In [53]:
variant_sh = gc.open_by_key("1_hKejRNMKYEf4jPQYSvzJZZUHIm-eNDDcuyc8LB6L5M")
variant_sheet = variant_sh.worksheet('DummyData')
variant_rows = variant_sheet.get_all_values()

variant_df = pd.DataFrame(variant_rows[1:], columns=variant_rows[0])

data = variant_df.loc[(variant_df.Gene == gene)]
variant_names = data['Variant'].to_numpy().tolist()
codon_names = data['Codon'].to_numpy().tolist()
# print(variant_names)
# print(codon_names)

i = 0
final_list = []
for i in range(len(variant_names)):
  variant = f"{variant_names[i]}|{codon_names[i]}"
  final_list.append(variant)

print(final_list)



['A146T|146', 'D153V|153', 'K147E|147', 'Y71H|71', 'A146V|146', 'N116H|116', 'T50I|50', 'Y40A|40', 'A146P|146', 'A59G|59', 'C118S|118', 'D154Q|154', 'D33E|33', 'E62K|62', 'F156L|156', 'F28L|28', 'G12A|12', 'G12C|12', 'G12D|12', 'G12F|12', 'G12R|12', 'G12S|12', 'G12V|12', 'G13C|13', 'G13D|13', 'G60R|60', 'K5N|5', 'L19F|19', 'N116S|116', 'P34L|34', 'P34R|34', 'Q22E|22', 'Q22K|22', 'Q22R|22', 'Q61H|61', 'Q61L|61', 'T58I|58', 'T74P|74', 'V14I|14', 'V14L|14', 'A66T|66', 'A66V|66', 'E31D|31', 'E63K|63', 'G12E|12', 'G12I|12', 'G12L|12', 'G12N|12', 'G12W|12', 'G12Y|12', 'G13V|13', 'G75E|75', 'K117N|117', 'Q61A|61', 'Q61E|61', 'Q61K|61', 'Q61P|61', 'Q61R|61', 'D57N|57', 'R164Q|164', 'A11P|11', 'A11T|11', 'A11V|11', 'A146G|146', 'A146X|146', 'A18D|18', 'A18V|18', 'A59E|59', 'A59S|59', 'A59T|59', 'A59X|59', 'D119N|119', 'D92Y|\xa0', 'E31K|31', 'E49K|49', 'E62G|62', 'G12X|12', 'G138V|138', 'G13A|13', 'G13E|13', 'G13R|13', 'G13S|13', 'G13X|13', 'G15S|15', 'G15X|15', 'G60D|60', 'G60E|60', 'H95D|95',

In [57]:
variant = "A146T|146"
variant, codon = variant.split("|")
print(variant)
# print(codon)/

A146T


In [ ]:
# update_spreadsheet(report_df, gene, "Report1")
report_df

In [ ]:
variant_sh = gc.open_by_key("1_hKejRNMKYEf4jPQYSvzJZZUHIm-eNDDcuyc8LB6L5M")
variant_sheet = variant_sh.worksheet('DummyData')
variant_rows = variant_sheet.get_all_values()

variant_df = pd.DataFrame(variant_rows[1:], columns=variant_rows[0])

data = variant_df.loc[(variant_df.Gene == gene)]
variant_names = data['Variant'].to_numpy().tolist()
codon_names = data['Codon'].to_numpy().tolist()
# print(variant_names)
codon_names

In [107]:
variant_sh = gc.open_by_key("1_hKejRNMKYEf4jPQYSvzJZZUHIm-eNDDcuyc8LB6L5M")
variant_sheet = variant_sh.worksheet('DummyData')
variant_rows = variant_sheet.get_all_values()

variant_df = pd.DataFrame(variant_rows[1:], columns=variant_rows[0])

variant_df = variant_df[variant_df['Impact'] == "duplication"]

data = variant_df.loc[(variant_df.Gene == gene)]

variants = data['Variant'].to_numpy().tolist()
# print(variants)
variant_names = []
for variant in variants:
  if "_" in variant:
    codon1, codon2 = variant.split("dup")[0].split("_")
    codon1 = codon1 + "dup"
    codon2 = codon2 + "dup"
    codon1 = codon1.split("dup")[0][1:]
    codon2 = codon2.split("dup")[0][1:]
    variant_name = f"{variant}|{codon1}|{codon2}"
    variant_names.append(variant_name)
  else:
    codon = variant.split("dup")[0][1:]
    variant_name = f"{variant}|{codon}"
    variant_names.append(variant_name)
variant_names

['A11_G12dup|11|12',
 'G10dup|10',
 'G12_G13dup|12|13',
 'G12dup|12',
 'G13dup|13',
 'exon dup|xon ']

In [ ]:
for variant in variant_names: #a, b
  if "_" in variant:
    for data in variant.split("|"):
      
  variant, codon = variant.split("|")
  codon = f"codon {codon}"
  codon = codon.lower()
  trial_rows=[]
  variant_found_in = []
  variants_found_list = []